## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
print(x_train.shape)

(50000, 32, 32, 3)


In [5]:
print(x_test.shape)

(10000, 32, 32, 3)


In [6]:
print(y_train.shape)

(50000, 10)


In [7]:
print(y_test.shape)

(10000, 10)


In [8]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3) ,input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3) ,activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100,activation='relu'))

#輸出
classifier.add(Dense(units=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=512,epochs=10)

Epoch 1/10
98/98 [==============================] - 35s 351ms/step - loss: 1.4467 - accuracy: 0.5355
Epoch 2/10
98/98 [==============================] - 35s 352ms/step - loss: 0.7560 - accuracy: 0.7415
Epoch 3/10
98/98 [==============================] - 35s 352ms/step - loss: 0.4429 - accuracy: 0.8510
Epoch 4/10
98/98 [==============================] - 34s 350ms/step - loss: 0.2418 - accuracy: 0.9270
Epoch 5/10
98/98 [==============================] - 35s 354ms/step - loss: 0.1149 - accuracy: 0.9732
Epoch 6/10
98/98 [==============================] - 35s 359ms/step - loss: 0.0472 - accuracy: 0.9939
Epoch 7/10
98/98 [==============================] - 36s 365ms/step - loss: 0.0198 - accuracy: 0.9988
Epoch 8/10
98/98 [==============================] - 37s 376ms/step - loss: 0.0091 - accuracy: 0.9999
Epoch 9/10
98/98 [==============================] - 36s 372ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 10/10
98/98 [==============================] - 36s 371ms/step - loss: 0.0036 - accura

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [9]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.9606565e-02, 3.1945663e-06, 7.9048090e-02, 8.3275902e-01,
        1.5806157e-02, 2.2533950e-06, 2.1339101e-05, 3.7481033e-04,
        2.3711685e-03, 7.4230425e-06]], dtype=float32)